<a href="https://colab.research.google.com/github/SujayVivek/TradingStrategies/blob/main/WorldQuant_BollingerBand_RSI_MACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install TA-Lib


  Using cached TA-Lib-0.4.28.tar.gz (357 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-Lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-Lib
Failed to build TA-Lib
ERROR: Could not build wheels for TA-Lib, which is required to install pyproject.toml-based projects


In [3]:
from google.colab import files
files.upload()

Saving 2015-2022 Nifty50 TimeSeries Data.csv to 2015-2022 Nifty50 TimeSeries Data.csv


{'2015-2022 Nifty50 TimeSeries Data.csv': b'\xef\xbb\xbfDate ,Open ,High ,Low ,Close ,Shares Traded ,Turnover (\xe2\x82\xb9 Cr)\r\n09-Nov-15,7788.25,7937.75,7771.7,7915.2,218422388,9376.17\r\n10-Nov-15,7877.6,7885.1,7772.85,7783.35,170267413,7153.47\r\n11-Nov-15,7838.8,7847.95,7819.1,7825,22380435,1123.44\r\n13-Nov-15,7762.45,7775.1,7730.9,7762.25,165876819,7731.55\r\n16-Nov-15,7732.95,7838.85,7714.15,7806.6,154134885,6871.15\r\n17-Nov-15,7848.75,7860.45,7793,7837.55,149451211,6367.14\r\n18-Nov-15,7823.15,7843.4,7725.05,7731.8,148037721,6112.32\r\n19-Nov-15,7788.5,7854.9,7765.45,7842.75,136702518,7410.15\r\n20-Nov-15,7841.9,7906.95,7817.8,7856.55,156610433,7298.01\r\n23-Nov-15,7869.5,7877.5,7825.2,7849.25,130871603,6099.09\r\n24-Nov-15,7837,7870.35,7812.65,7831.6,134562905,6686.27\r\n26-Nov-15,7837.15,7897.1,7832,7883.8,226632193,11054.04\r\n27-Nov-15,7910.6,7959.3,7879.45,7942.7,154863047,7068.72\r\n30-Nov-15,7936.25,7966,7922.8,7935.25,223041699,15368.67\r\n01-Dec-15,7958.15,7972.15,

In [7]:
import pandas as pd
import numpy as np

# Sample DataFrame with Open , Close, High, Low prices
data = pd.read_csv('2015-2022 Nifty50 TimeSeries Data.csv')

# Calculate MACD and SIGNAL
short_window = 12
long_window = 26
signal_window = 9
data['ShortEMA'] = data['Close '].ewm(span=short_window, adjust=False).mean()
data['LongEMA'] = data['Close '].ewm(span=long_window, adjust=False).mean()
data['MACD'] = data['ShortEMA'] - data['LongEMA']
data['SIGNAL'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()

# Calculate RSI
rsi_window = 14
delta = data['Close '].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=rsi_window).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_window).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

# Calculate ADX
adx_window = 14
data['TR'] = np.maximum(np.maximum(data['High '] - data['Low '], abs(data['High '] - data['Close '].shift())), abs(data['Low '] - data['Close '].shift()))
data['DMplus'] = np.where((data['High '] - data['High '].shift()) > (data['Low '].shift() - data['Low ']), np.maximum(data['High '] - data['High '].shift(), 0), 0)
data['DMminus'] = np.where((data['Low '].shift() - data['Low ']) > (data['High '] - data['High '].shift()), np.maximum(data['Low '].shift() - data['Low '], 0), 0)
atr = data['TR'].rolling(window=adx_window).mean()
data['DIplus'] = 100 * (data['DMplus'].rolling(window=adx_window).mean() / atr)
data['DIminus'] = 100 * (data['DMminus'].rolling(window=adx_window).mean() / atr)
data['DX'] = 100 * (np.abs(data['DIplus'] - data['DIminus']) / (data['DIplus'] + data['DIminus']))
data['ADX'] = data['DX'].rolling(window=adx_window).mean()

# Calculate Bollinger Bands
bb_window = 20
data['SMA'] = data['Close '].rolling(window=bb_window).mean()
data['STD'] = data['Close '].rolling(window=bb_window).std()
data['UPBAND'] = data['SMA'] + (data['STD'] * 2)
data['LOWBAND'] = data['SMA'] - (data['STD'] * 2)

print(data)


          Date      Open      High       Low     Close   Shares Traded   \
0     09-Nov-15   7788.25   7937.75   7771.70   7915.20     218422388.0   
1     10-Nov-15   7877.60   7885.10   7772.85   7783.35     170267413.0   
2     11-Nov-15   7838.80   7847.95   7819.10   7825.00      22380435.0   
3     13-Nov-15   7762.45   7775.10   7730.90   7762.25     165876819.0   
4     16-Nov-15   7732.95   7838.85   7714.15   7806.60     154134885.0   
...         ...       ...       ...       ...       ...             ...   
1517  27-Dec-21  16937.75  17112.05  16833.20  17086.25     144777457.0   
1518  28-Dec-21  17177.60  17250.25  17161.15  17233.25     176026100.0   
1519  29-Dec-21  17220.10  17285.95  17176.65  17213.60     161679423.0   
1520  30-Dec-21  17201.45  17264.05  17146.35  17203.95     320831676.0   
1521  31-Dec-21  17244.50  17400.80  17238.50  17354.05     167025720.0   

      Turnover (₹ Cr)      ShortEMA       LongEMA        MACD  ...  DMplus  \
0             9376.17

In [10]:
def device_strategy(data,cash):
    x = data.dropna().copy()
    x = (x.reset_index())
    pos = 'None'
    macd_cross = 'None'
    pnl = []
    stop = 0
    longs = 0
    short = 0
    brokerage_per = 0.0045#Brokerage per share trade
    trades = 0

    for i in range(0,len(x)):
        pct_change = 0
        #Taking Positions===============================================
        #Identify MACD and signal line crossover and set indicator
        if x.iloc[i-1]['MACD'] < x.iloc[i-1]['SIGNAL']\
        and x.iloc[i]['MACD'] > x.iloc[i]['SIGNAL']:
            macd_cross = 'Up'
        if x.iloc[i-1]['MACD'] > x.iloc[i-1]['SIGNAL']\
        and x.iloc[i]['MACD'] < x.iloc[i]['SIGNAL']:
            macd_cross = 'Down'

        #When no position is held try entering trades
        if pos == 'None' and i > 1:
                #Slippage
                d1 = x.iloc[i]['Open '] - x.iloc[i-1]['Close ']
                d2 = x.iloc[i]['High '] - x.iloc[i-1]['Close ']
                if d2 == 0:
                    d2 = d1

                #1. Go Long strategy
                #¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬
                if x.iloc[i-1]['RSI'] <= 30 and x.iloc[i]['RSI'] > 30:#RSI cross 30 from below
                    if macd_cross == 'Down' and x.iloc[i]['MACD'] < x.iloc[i]['SIGNAL']:
                        if x.iloc[i]['ADX'] > 25 and x.iloc[i]['Index ADX'] > 25:
                            if x.iloc[i-1]['Close '] < x.iloc[i-1]['LOWBAND']:#Candle-Stick Close below Lower Bollinger Band
                                #Go Long-----------------------------------
                                pos = 'Long'
                                P = x.iloc[i-1]['Close ']
                                Slippage = (d1 / d2)/100
                                P = P + (P*Slippage) + brokerage_per
                                position = cash/P
                                cash = cash - (P*position)
                                trades+=1
                                longs+=1

                #1. Go Short strategy
                #¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬
                if x.iloc[i-1]['RSI'] >= 70 and x.iloc[i]['RSI'] < 70:#RSI cross 70 from above
                    if macd_cross == 'Up' and x.iloc[i]['MACD'] > x.iloc[i]['SIGNAL']:
                        if x.iloc[i]['ADX'] > 25 and x.iloc[i]['Index ADX'] > 25:
                            if x.iloc[i-1]['Close '] > x.iloc[i-1]['UPBAND']:#Candle-Stick Close above Upper Bollinger Band
                                #Go Short-----------------------------------
                                pos = 'Short'
                                S = x.iloc[i-1]['Close ']
                                Slippage = (d1 / d2)/100
                                S = S + (S*Slippage) - brokerage_per
                                position = cash/S
                                cash = cash + (S*position)
                                trades+=1
                                short+=1

        #Exit Strategy¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬
        if pos == 'Long':
            stop_down = P - (P*0.05)
            #Lock Profit on Long
            if x.iloc[i-1]['Close '] >= x.iloc[i]['UPBAND']:
                #Exit Position by locking profit------------------
                pos = 'None'
                S = x.iloc[i-1]['Close ']
                Slippage = (d1 / d2)/100
                S = S + (S*Slippage) - brokerage_per
                cash = cash + (S*position)
                pct_change =  (S/P-1)
                position = 0
            #Stop Loss execution on Long
            elif P > x.iloc[i-1]['Close ']:
                if x.iloc[i-1]['Close '] <= stop_down:
                    #Exit Position by Stop Loss-------------------------
                    pos = 'None'
                    Slippage = (d1 / d2)/100
                    SL = stop_down + (stop_down*Slippage) - brokerage_per
                    pct_change =  (SL/P-1)
                    cash = cash + (SL*position)
                    position = 0
                    stop+=1
        if pos == 'Short':
            #Lock Profit on Short
            stop_up = S + (S*0.05)
            if x.iloc[i-1]['Close '] <= x.iloc[i]['LOWBAND']:
                #Exit Position by locking profit--------------------
                pos = 'None'
                P = x.iloc[i-1]['Close ']
                Slippage = (d1 / d2)/100
                P = P + (P*Slippage) + brokerage_per
                cash = cash - (P*position)
                pct_change =  (S/P-1)
                position = 0
            #Stop Loss execution on Short
            elif S < x.iloc[i-1]['Close ']:
                if x.iloc[i-1]['Close '] >= stop_up:
                    #Exit Position by Stop Loss--------------------------
                    pos = 'None'
                    Slippage = (d1 / d2)/100
                    SL = stop_up + (stop_up*Slippage) + brokerage_per
                    pct_change =  (S/SL-1)
                    cash = cash - (SL*position)
                    position = 0
                    stop+=1
        #Exit on End of the trade
        if i == len(x) - 1 and pos == 'None':
            E = x.iloc[i-1]['Close ']
            if pos == 'Long':
               Slippage = (d1 / d2)/100
               E = E + (E*Slippage) - brokerage_per
               pct_change =  (E/P-1)
               cash = cash + (E*position)
            if pos == 'Short':
               Slippage = (d1 / d2)/100
               E = E + (E*Slippage) + brokerage_per
               pct_change =  (S/E-1)
               cash = cash - (E*position)
            position = 0
            pos = 'None'

        pnl.append(pct_change)
    x['PnL'] = pnl#Append computed PnL

    return x,trades,longs,short,stop

In [13]:
import matplotlib.pyplot as plt

# Assuming df is your Nifty 50 historical DataFrame

# Call device_strategy function
data, trades, longs, shorts, stops = device_strategy(data, cash=100000)

# Plot the graphs
plt.figure(figsize=(14, 7))

# Plot Close price
plt.plot(data['Close '], label='Close Price', color='black')

# Plot long entry points
plt.plot(data[data['Long'] == 1].index, data['Close '][data['Long'] == 1], '^', markersize=10, color='g', lw=0, label='Long')

# Plot short entry points
plt.plot(data[data['Short'] == 1].index, data['Close '][data['Short'] == 1], 'v', markersize=10, color='r', lw=0, label='Short')

plt.title('Nifty 50 Historical Data with Long and Short Positions')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


KeyError: 'Open'